In [ ]:
# Importing pandas library
import pandas as pd
# Reading file 
star_wars_8_df = pd.read_csv("/content/INST737_Star_Wars_IMDB_StarWars8.csv")

In [ ]:
star_wars_8_df.head()

,Unnamed: 0,Review_Date,Author,Rating,Review_Title,Review,Review_Url
0,0,24 December 2017,BA_Harrison,3.0,I'm going to keep this short (unlike the film...,After George Lucas's execrable prequels and th...,/review/rw5308106/?ref_=tt_urv
1,1,16 December 2017,view_and_review,3.0,Not this Disappointed Since Jar Jar Binks\n,I love the Star Wars franchise. I was so anxio...,/review/rw5308106/?ref_=tt_urv
2,2,21 January 2018,gogoschka-1,NaN,My Issues With The Storytelling in The Last J...,"I didn't hate TLJ, but even if I completely ig...",/review/rw5308106/?ref_=tt_urv
3,3,19 April 2021,mn2000,1.0,An embarrassment for the Star Wars fans\n,"When I watch this movie, it only makes me reme...",/review/rw5308106/?ref_=tt_urv
4,4,17 December 2017,marc5477,1.0,What the hell was this?\n,Wow this is getting hammered by reviews and no...,/review/rw5308106/?ref_=tt_urv


In [ ]:
# Removing unnecessary columns
star_wars_8_df_v2 = star_wars_8_df.drop(columns = ["Review_Date", "Author", "Rating", "Review_Title", "Review_Url", "Unnamed: 0"])

In [ ]:
star_wars_8_df_v2

,Review
0,After George Lucas's execrable prequels and th...
1,I love the Star Wars franchise. I was so anxio...
2,"I didn't hate TLJ, but even if I completely ig..."
3,"When I watch this movie, it only makes me reme..."
4,Wow this is getting hammered by reviews and no...
...,...
7081,I love how the main music theme is based upon ...
7082,Episode 8 was a very interesting movie. It was...
7083,"No spoilers, but If you haven't seen The Last ..."
7084,I'd describe The Last Jedi as a bad good movie...


In [ ]:
# Topic Modeling Section


In [ ]:
# Installing Topic Modeling Libraries
!pip install regex
!pip install numpy
!pip install gensim
!pip install spacy
!pip install pyLDAvis
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=a012d47c598ace9d3d11990b7bccab75a5652e3aae87a3650c27a17e2f390165
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created whe

In [ ]:
# Importing Necessary Libraries
import regex
import numpy as np
import pandas as pd

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import nltk
import regex as re
nltk.download('stopwords')

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
# Removing emails and newline characters
data = star_wars_8_df_v2['Review'].tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', str(sent)) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', str(sent)) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", str(sent)) for sent in data]

print(data[0])

<>:5: DeprecationWarning: invalid escape sequence \S
<>:8: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \S
<>:8: DeprecationWarning: invalid escape sequence \s
<ipython-input-10-df5f02afbe48>:5: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', '', str(sent)) for sent in data]
<ipython-input-10-df5f02afbe48>:8: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', str(sent)) for sent in data]


After George Lucass execrable prequels and the mediocre The Force Awakens, Ive learnt to keep my expectations low for any new Star Wars films in order to avoid total disappointment. Having just seen The Last Jedi, it appears that I havent been keeping them low enough.


In [ ]:
#Tokenize words and Clean up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
data_words = list(sent_to_words(data))

print(data_words[0])

['after', 'george', 'lucass', 'execrable', 'prequels', 'and', 'the', 'mediocre', 'the', 'force', 'awakens', 'ive', 'learnt', 'to', 'keep', 'my', 'expectations', 'low', 'for', 'any', 'new', 'star', 'wars', 'films', 'in', 'order', 'to', 'avoid', 'total', 'disappointment', 'having', 'just', 'seen', 'the', 'last', 'jedi', 'it', 'appears', 'that', 'havent', 'been', 'keeping', 'them', 'low', 'enough']


In [ ]:
!python -m spacy download en_core_web_sm


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-11 00:33:54.639407: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 2.7 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
# Build the bigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

print(bigram_mod[data_words[4]])

['wow', 'this', 'is', 'getting', 'hammered', 'by', 'reviews', 'and', 'now', 'understand', 'why', 'given', 'its', 'still', 'in', 'week', 'and', 'most', 'of', 'the', 'early', 'reviews', 'were', 'all', 'paid', 'for', 'and', 'fake', 'the', 'fact', 'that', 'this', 'is', 'at']


In [ ]:
# Remove Stopwords and Lemmatize
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)

# nlp = spacy.load("en_core_web_sm").load()
# nlp = spacy.load('en', disable=['parser', 'ner'])
# python -m spacy download en_core_web_sm

# nlp = en_core_web_sm.load()
nlp = spacy.load("en_core_web_sm")

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0])

['execrable', 'prequel', 'mediocre', 'force_awaken', 'learn', 'keep', 'expectation', 'low', 'new', 'star', 'war', 'film', 'order', 'avoid', 'total', 'disappointment', 'see', 'appear', 'keep', 'low', 'enough']


In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1)]


In [ ]:
!pip install -U gensim
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
# Print the KeywordS in the 20 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 24.3 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  self.update_eta(self.state.get_lambda(), rho)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  self.update_eta(self.state.get_lambda(), rho)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  self.update_eta(self.state.get_lambda(), rho)
/usr/local/lib/python3.8/dist-packages/gensim/models/ld

[(0, '0.056*"force" + 0.033*"snoke" + 0.028*"rey" + 0.027*"kill" + 0.022*"luke" + 0.022*"ship" + 0.021*"space" + 0.019*"fight" + 0.019*"die" + 0.017*"resistance"'), (1, '0.055*"opinion" + 0.034*"beautiful" + 0.024*"d" + 0.023*"compare" + 0.022*"criticism" + 0.018*"silly" + 0.017*"human" + 0.016*"porg" + 0.016*"entirely" + 0.014*"remake"'), (2, '0.019*"amazing" + 0.018*"mean" + 0.018*"battle" + 0.017*"bring" + 0.015*"however" + 0.014*"always" + 0.010*"seem" + 0.009*"theme" + 0.009*"hard" + 0.008*"definitely"'), (3, '0.052*"interesting" + 0.047*"idea" + 0.024*"surprise" + 0.023*"half" + 0.021*"rebel" + 0.020*"break" + 0.019*"escape" + 0.018*"major" + 0.017*"chase" + 0.015*"buy"'), (4, '0.040*"critic" + 0.036*"follow" + 0.032*"other" + 0.025*"awesome" + 0.024*"deep" + 0.023*"open" + 0.023*"close" + 0.022*"fresh" + 0.021*"include" + 0.020*"laugh"'), (5, '0.060*"feel" + 0.052*"time" + 0.039*"go" + 0.033*"come" + 0.031*"original" + 0.030*"first" + 0.027*"trilogy" + 0.026*"leave" + 0.022*"bac

/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  self.update_eta(self.state.get_lambda(), rho)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  self.update_eta(self.state.get_lambda(), rho)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  self.update_eta(self.state.get_lambda(), rho)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(

In [ ]:
# Visualize the models
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14    -0.349967  0.003287       1        1  28.426424
6     -0.337668 -0.006045       2        1  23.165698
5     -0.209869  0.085232       3        1  10.895572
2     -0.016929 -0.418353       4        1   8.830907
0     -0.052652  0.148766       5        1   7.043098
10     0.076315  0.034456       6        1   3.018669
13     0.086666  0.016462       7        1   2.499738
7      0.087049  0.017453       8        1   2.437462
3      0.092947  0.019359       9        1   2.388107
4      0.091114  0.017543      10        1   2.321977
8      0.102257  0.006650      11        1   1.972264
11     0.102738  0.019269      12        1   1.870564
12     0.105176  0.018074      13        1   1.857312
9      0.105723  0.019187      14        1   1.818754
1      0.117100  0.018661      15        1   1.453453, topic_info=           Term         Freq        Total Category  logprob  loglift
18          war  5815.000000  5815.000000  Default  30.0000  30.0000
16         star  5541.000000  5541.000000  Default  29.0000  29.0000
58        movie  9101.000000  9101.000000  Default  28.0000  28.0000
159        feel  1727.000000  1727.000000  Default  27.0000  27.0000
32    character  2955.000000  2955.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
2340       lady    22.795511    23.610158  Topic15  -5.1254   4.1961
3287       baby    20.906321    21.720965  Topic15  -5.2119   4.1930
4215   ordinary    20.746702    21.561348  Topic15  -5.2195   4.1927
2512      image    20.549670    21.364315  Topic15  -5.2291   4.1924
5329      tacky    20.662162    21.492971  Topic15  -5.2236   4.1918

[534 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
3325     12  0.987563      ability
928       1  0.998270          act
311       1  0.998654       action
679       6  0.997649        actor
5472     12  0.968174  adam_driver
...     ...       ...          ...
773       7  0.992589       writer
481       2  0.999229         year
779      13  0.980748    yesterday
1148      3  0.998126          yet
213       8  0.988442        young

[566 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 7, 6, 3, 1, 11, 14, 8, 4, 5, 9, 12, 13, 10, 2])